In [5]:
import os
import torch
from torchvision import transforms, models
from PIL import Image
import mediapipe as mp
import numpy as np
import json
import pandas as pd
import joblib
from facenet_pytorch import MTCNN
import warnings
from sklearn.exceptions import NotFittedError
from difflib import get_close_matches
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# ----------------------------
# Pose-Guided CNN (Body Type)
# ----------------------------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils  # For drawing pose landmarks

def extract_pose_keypoints(image):
    """Extracts pose keypoints from an image using MediaPipe."""
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(np.array(image))
        if not results.pose_landmarks:
            return np.zeros(33 * 3), None
        keypoints = []
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.visibility])
        return np.array(keypoints, dtype=np.float32), results.pose_landmarks

class PoseGuidedCNN(torch.nn.Module):
    """CNN model for body type classification using pose keypoints."""
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.efficientnet_b0(pretrained=False)
        self.backbone.classifier = torch.nn.Identity()
        self.pose_fc = torch.nn.Sequential(
            torch.nn.Linear(33 * 3, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(128 + 1280, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.fc_out = torch.nn.Linear(512, num_classes)

    def forward(self, x_img, x_pose):
        img_feats = self.backbone(x_img)
        pose_feats = self.pose_fc(x_pose)
        combined = torch.cat([img_feats, pose_feats], dim=1)
        att_feats = self.attention(combined)
        return self.fc_out(att_feats)

def load_class_mapping(mapping_path):
    """Loads a class mapping from a JSON file."""
    try:
        with open(mapping_path, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"❌ Error: Class mapping file '{mapping_path}' not found.")
        return {}

def get_gender_classes(class_names, gender):
    """Filters class names by gender."""
    indices = [i for i, name in enumerate(class_names) if name.startswith(gender.lower() + "/")]
    names = [class_names[i] for i in indices]
    return indices, names

def predict_body_type(image_path, model, class_names, device, gender):
    """Predicts the body type from an image."""
    gender_indices, gender_class_names = get_gender_classes(class_names, gender)
    if not gender_indices:
        print(f"❌ Error: No body types found for gender '{gender}'.")
        return None, None
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"❌ Error: Image file '{image_path}' not found.")
        return None, None

    pose_feats, pose_landmarks = extract_pose_keypoints(image)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)
    pose_tensor = torch.tensor(pose_feats, dtype=torch.float).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(img_tensor, pose_tensor)
        outputs_gender = outputs[:, gender_indices]
        _, pred = torch.max(outputs_gender, 1)
    return gender_class_names[pred.item()], pose_landmarks

# ----------------------------
# Plot image with pose landmarks and prediction
# ----------------------------
def plot_image_with_pose(image_path, pose_landmarks, pred_class):
    """Plots the image with pose landmarks and prediction title."""
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)
    
    # Draw pose landmarks
    if pose_landmarks:
        annotated_image = image_np.copy()
        mp_drawing.draw_landmarks(
            annotated_image,
            pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )
    else:
        annotated_image = image_np
    
    # Display the image with prediction as title
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_image)
    plt.title(f"Predicted Body Type: {pred_class}", fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# ----------------------------
# Skin Tone Model
# ----------------------------
def ordinal_to_class(ordinal_logits):
    """Converts ordinal logits to a single class prediction."""
    prob = torch.sigmoid(ordinal_logits)
    return torch.sum(prob > 0.5, dim=1)

class HybridSkinToneModel(torch.nn.Module):
    """Model for predicting skin tone."""
    def __init__(self, num_classes=6, use_uncertainty=True):
        super().__init__()
        self.backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        feat_dim = self.backbone.classifier[1].in_features
        self.backbone.classifier = torch.nn.Identity()
        self.ordinal_head = torch.nn.Linear(feat_dim, num_classes - 1)
        self.regression_head = torch.nn.Linear(feat_dim, 1)
        self.use_uncertainty = use_uncertainty
        if use_uncertainty:
            self.uncertainty_head = torch.nn.Linear(feat_dim, 1)

    def forward(self, x):
        features = self.backbone(x)
        ordinal_logits = self.ordinal_head(features)
        reg_pred = self.regression_head(features)
        logvar = self.uncertainty_head(features) if self.use_uncertainty else None
        return ordinal_logits, reg_pred, logvar

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

class_labels_skin = ['brown', 'dark', 'light', 'medium', 'olive', 'verylight']

def predict_skin_tone(image_path, device, model):
    """Predicts skin tone from a detected face in an image."""
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"❌ Error: Image file '{image_path}' not found.")
        return None
    mtcnn = MTCNN(keep_all=False, device=device)
    face_tensor = mtcnn(image)
    if face_tensor is None:
        print("⚠️ No face detected.")
        return None
    face_pil = transforms.ToPILImage()(face_tensor.cpu())
    input_tensor = transform_test(face_pil).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        ordinal_logits, _, _ = model(input_tensor)
        predicted_class = ordinal_to_class(ordinal_logits).item()
    return class_labels_skin[predicted_class]

# ----------------------------
# Fashion Stylist AI (Enhanced)
# ----------------------------
class FashionStylistAI:
    """AI for outfit advice based on gender, skin tone, body type, and dress type."""
    def __init__(self):
        self.models = {}
        self.encoders = {}
        self.is_trained = False
        self.input_features = ['Gender', 'SkinTone', 'BodyType', 'DressType']
        self.output_features = [
            'Fabric', 'Neckline', 'SleeveLength', 'RecommendedColors',
            'IdealAccessories', 'Footwear', 'MakeupAccent', 'HairStyle'
        ]
        self.raw_data = None

    def load_raw_data(self, filepath):
        """Load and normalize CSV data."""
        try:
            self.raw_data = pd.read_csv(filepath, on_bad_lines='skip', engine='python')
            self.raw_data.columns = [col.lower() for col in self.raw_data.columns]
            for col in self.raw_data.columns:
                if self.raw_data[col].dtype == 'object':
                    self.raw_data[col] = self.raw_data[col].str.strip().str.lower()
            print("Raw fashion data loaded and normalized successfully.")
        except FileNotFoundError:
            print(f"❌ Error: The raw data file '{filepath}' was not found.")
            self.raw_data = None

    def _predict_features(self, gender, skin_tone, body_type, dress_type):
        if not self.is_trained:
            if not self.models or not self.encoders:
                print("❌ Error: The loaded model seems incomplete.")
                return None
            self.is_trained = True
        input_data = pd.DataFrame(
            [[gender.title(), skin_tone.title(), body_type.title(), dress_type.title()]],
            columns=self.input_features
        )
        input_encoded = pd.DataFrame()
        try:
            for col in self.input_features:
                encoder = self.encoders[col]
                known_classes = set(encoder.classes_)
                if input_data[col].iloc[0] not in known_classes:
                    input_encoded[col] = [0]
                else:
                    input_encoded[col] = encoder.transform(input_data[col])
        except Exception as e:
            print(f"❌ Error encoding input: {e}")
            return None
        recommendation_facts = {}
        for feature, model in self.models.items():
            prediction_encoded = model.predict(input_encoded)[0]
            recommendation_facts[feature] = self.encoders[feature].inverse_transform([prediction_encoded])[0]
        return recommendation_facts

    def generate_recommendation(self, gender, skin_tone, body_type, dress_type=None):
        """Generates list of suitable dress types or detailed recommendation."""
        if self.raw_data is None:
            return "Cannot provide suggestions because the original data file was not loaded."

        # Helper: closest match from CSV
        def normalize_match(value, possible_values):
            matches = get_close_matches(value.lower(), [v.lower() for v in possible_values], n=1, cutoff=0.5)
            return matches[0] if matches else value.lower()

        gender_norm = normalize_match(gender, self.raw_data['gender'].unique())
        skin_tone_norm = normalize_match(skin_tone, self.raw_data['skintone'].unique())
        body_type_norm = normalize_match(body_type, self.raw_data['bodytype'].unique())

        if dress_type is None:
            query = (
                self.raw_data['gender'].str.lower() == gender_norm
            ) & (
                self.raw_data['skintone'].str.lower() == skin_tone_norm
            ) & (
                self.raw_data['bodytype'].str.lower() == body_type_norm
            )
            filtered_df = self.raw_data[query]
            if filtered_df.empty:
                print("⚠️ Exact match not found. Trying relaxed filter...")
                query = (
                    self.raw_data['gender'].str.lower() == gender_norm
                ) & (
                    self.raw_data['bodytype'].str.lower() == body_type_norm
                )
                filtered_df = self.raw_data[query]
            return filtered_df['dresstype'].unique().tolist()

        # Detailed recommendation if dress_type provided
        facts = self._predict_features(gender, skin_tone, body_type, dress_type)
        if not facts:
            return "Could not generate a recommendation due to an error in prediction."
        recommendation = {
            'Overall Style & Fit':
                f"For a {gender} with a {body_type} body type, the goal is to create "
                f"a balanced and flattering silhouette. We recommend a style with a "
                f"{facts.get('SleeveLength', 'suitable')} sleeve length to complement the look.",
            'Neckline & Fabric':
                f"A {facts.get('Neckline', 'flattering')} neckline will work wonderfully. "
                f"For fabric, consider {facts.get('Fabric', 'a comfortable material')} "
                f"as it drapes well and provides a great fit.",
            'Color Palette':
                f"With your {skin_tone} skin tone, colors like {facts.get('RecommendedColors', 'versatile shades')} "
                f"will be particularly striking.",
            'Accessorizing & Finishing Touches':
                f"Complete your outfit with {facts.get('IdealAccessories', 'beautiful accessories')} "
                f"and a pair of {facts.get('Footwear', 'stylish footwear')}. For makeup, "
                f"a {facts.get('MakeupAccent', 'subtle accent')} will tie everything together, "
                f"and a {facts.get('HairStyle', 'chic hairstyle')} will be the perfect finishing touch."
        }
        return recommendation

    @classmethod
    def load_model(cls, filepath):
        try:
            print(f"🔄 Loading predictive model from {filepath}...")
            model = joblib.load(filepath)
            model.is_trained = True
            print("Model loaded successfully.")
            return model
        except FileNotFoundError:
            print(f"❌ Error: Model file not found at {filepath}")
            return None

# ----------------------------
# Full Pipeline Execution
# ----------------------------
def full_style_recommendation(image_path, gender):
    """Runs full pipeline from image analysis to style recommendation."""
    print("🚀 Starting the Fashion Style Recommendation Pipeline...")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Body Type model
    try:
        class_mapping = load_class_mapping("saved_models/class_mapping.json")
        model_bodytype = PoseGuidedCNN(num_classes=len(class_mapping))
        model_bodytype.load_state_dict(torch.load("saved_models/bodytype_fold1.pth", map_location=device))
        model_bodytype.to(device)
    except Exception as e:
        print(f"❌ Error loading Body Type model: {e}")
        return

    # Load Skin Tone model
    try:
        model_skin = HybridSkinToneModel()
        model_skin.load_state_dict(torch.load("saved_models1/hybrid_skin_tone_model.pth", map_location=device))
        model_skin.to(device)
    except Exception as e:
        print(f"❌ Error loading Skin Tone model: {e}")
        return

    # Predict body type and skin tone
    print("🕵️‍♀️ Analyzing image for body type and skin tone...")
    body_type_pred, pose_landmarks = predict_body_type(image_path, model_bodytype, class_mapping, device, gender)
    skin_tone_pred = predict_skin_tone(image_path, device, model_skin)

    if body_type_pred is None or skin_tone_pred is None:
        print("Cannot generate recommendation due to failed prediction.")
        return

    body_type_pred_norm = body_type_pred.split('/')[-1].strip().lower()
    skin_tone_pred_norm = skin_tone_pred.lower()
    gender_norm = gender.lower()

    print(f"✅ Predicted Body Type: {body_type_pred_norm}, Predicted Skin Tone: {skin_tone_pred_norm}")

    # Plot image with pose landmarks and prediction
    plot_image_with_pose(image_path, pose_landmarks, body_type_pred_norm)

    # Load Fashion Stylist AI
    stylist_ai = FashionStylistAI.load_model("fashion_model_cpu.joblib")
    if stylist_ai is None:
        print("❌ Could not load the detailed stylist model. Only general recommendations will be possible.")
        return

    stylist_ai.load_raw_data("fashiondata.csv")

    # Generate list of suitable dress types
    print("\n💡 Generating a list of suitable dress types...")
    suitable_dress_types = stylist_ai.generate_recommendation(gender_norm, skin_tone_pred_norm, body_type_pred_norm)
    if not suitable_dress_types:
        print("Sorry, no suitable dress types were found in the database for your profile.")
        return

    print("Found suitable dress types:", suitable_dress_types)

    # Detailed recommendation for first dress type
    print("\n✍️ Generating a detailed style guide for the first suitable dress type found...")
    chosen_dress_type = suitable_dress_types[0]
    detailed_recommendation = stylist_ai.generate_recommendation(
        gender_norm, skin_tone_pred_norm, body_type_pred_norm, dress_type=chosen_dress_type
    )

    if isinstance(detailed_recommendation, dict):
        print("\n✨ Your Personalized Style Guide for a " + chosen_dress_type.capitalize() + " ✨")
        print("--------------------------------------------------")
        for section, advice in detailed_recommendation.items():
            print(f"**{section}**\n{advice}\n")
        print("--------------------------------------------------")
    else:
        print("❌ Could not generate a detailed recommendation.")
        print(detailed_recommendation)

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    test_imagetest/triangle.p"
    gender = "male"  # Change to "female" for female test image

    if os.path.exists(test_image):
        full_style_recommendation(test_image, gender)
    else:
        print(f"❌ Error: Test image '{test_image}' not found.")
        print("Please ensure you have a test image in the specified path.")

SyntaxError: unterminated string literal (detected at line 397) (1818971202.py, line 397)